## Requirement

(Optional) Using Generation by Syntax Tree and PAIP 151-167. Generate Sentence by Grammar. Implement a dialogue system, you may implement a mock system like West World, or Prison Break, etc.

- Using AIML markup language to generate same pattern. 

- Think what markup language or rule based can do, what they cannot do?

Bonus: If you finish this task, please connect with me. I will send you a **mysterious gift**.

### References:
a. PAIP https://github.com/Artificial-Intelligence-for-NLP/References/blob/master/AI%20%26%20Machine%20Learning/Paradigms-of-Artificial-Intelligence-Programming%20copy.pdf

b. https://www.wikiwand.com/en/ELIZA

c. https://www.eclecticenergies.com/ego/eliza

d. https://baike.baidu.com/item/%E8%89%BE%E4%B8%BD%E8%8E%8E/9030600

# chapter 5: ELIZA: Dialog with a Machine
## 5.1 Describing and Specifying ELIZA
Now that we have an idea of what ELIZA is like, we can begin the description and spec- ificationof the program, and eventually move to the implementation and debugging. The ELIZA algorithm can be described simply as: 

- (1)read an input, 

- (2) find a pattern that matches the input, 

- (3)transform the input into a response, and 

- (4)print the response. These four steps are repeated for each input.

The specificationand implementation of steps (1)and (4)are trivial: for (I),use the built-in read function to read a list of words, and for (4)use pri n t to print the list of words in the response.
Of course, there are some drawbacks to this specification. The user will have to type a real list- using parentheses- and the user can't use characters that are special to read, like quotation marks, commas, and periods. So our input won't be as unconstrained as in the sample dialog, but that's a small price to pay for the convenience of having half of the problem neatly solved.


## 1.Rules

In [1]:
eliza_rules = '''
(?* ?x) hello (?* ?y)
    how do you do. Please state your problem.
(?* ?x) i want (?* ?y)
    what would it mean if you got ?y
    why do you want ?y
(?* ?x) if (?* ?y)
    do you really think its likely that ?y
    what do you think about ?y
    really-- if ?y
(?* ?x) no (?* ?y)
    why not?
    you are being a bit negative
    are you saying "NO" just to be negative?
(?* ?x) i feel (?* ?y)
    do you often feel ?y ?
(?* ?x) i felt (?* ?y)
    what other feelings do you have?
'''

from collections import defaultdict
pattern_to_template = defaultdict(list)
eliza_rules = eliza_rules.splitlines()
for line in eliza_rules:
    if not line: 
        continue
    if line[0] != ' ':
        key = line
    else:
        pattern_to_template[key].append(line.strip())
        
pattern_to_template

defaultdict(list,
            {'(?* ?x) hello (?* ?y)': ['how do you do. Please state your problem.'],
             '(?* ?x) i feel (?* ?y)': ['do you often feel ?y ?'],
             '(?* ?x) i felt (?* ?y)': ['what other feelings do you have?'],
             '(?* ?x) i want (?* ?y)': ['what would it mean if you got ?y',
              'why do you want ?y'],
             '(?* ?x) if (?* ?y)': ['do you really think its likely that ?y',
              'what do you think about ?y',
              'really-- if ?y'],
             '(?* ?x) no (?* ?y)': ['why not?',
              'you are being a bit negative',
              'are you saying "NO" just to be negative?']})

### pattern matching v1
```lisp
(defun pat-match (pattern input)
  (if (variable-p pattern)
    t
    (if (or (atom pattern ) (atom input))
      (eql pattern input)
      (and (pat-match (car pattern) (car input))
           (pat-match (cdr pattern) (cdr input))))))           
           
(defun variable-p (x)
  (and (symbolp x) (equal (char (symbol-name x) 0) #\?)))
```
**input** is the key word of python, here use **_input**

In [32]:
import debug_tools

# @debug_tools.debug_print
def atom(x):
    return type(x) == str

# @debug_tools.debug_print
def variable_p(x):
    return x[0] == '?'

@debug_tools.debug_print
def pat_match(pattern:list, _input:list):
    #"Does pattern match input? Any variable can match anything."
    if (pattern == [] and _input == []) or variable_p(pattern):
        return True
    else:
        if atom(pattern) or atom(_input):
            return pattern == _input
        else:
            return pat_match(pattern[0], _input[0]) and pat_match(pattern[1:], _input[1:])

test_cases = """
I need a ?X <-> I need a vacation
I need a ?X <-> I need two vacations
"""

def test_pat_match(pat_match, test_cases):


    debug_tools.stack_level = 0
    for test_case in test_cases.split('\n'):
        if not test_case: continue
        pattern, _input = [s.strip().split() for s in test_case.split('<->')]
        print('\n=====================================================================')
        print('pattern:\t', pattern)
        print('input:\t', _input)
        print(pat_match(pattern, _input))

test_pat_match(pat_match, test_cases)


pattern:	 ['I', 'need', 'a', '?X']
input:	 ['I', 'need', 'a', 'vacation']
|s: pat_match ((['I', 'need', 'a', '?X'], ['I', 'need', 'a', 'vacation']), {})
||s: pat_match (('I', 'I'), {})
||e: pat_match (('I', 'I'), {}) = True
||s: pat_match ((['need', 'a', '?X'], ['need', 'a', 'vacation']), {})
|||s: pat_match (('need', 'need'), {})
|||e: pat_match (('need', 'need'), {}) = True
|||s: pat_match ((['a', '?X'], ['a', 'vacation']), {})
||||s: pat_match (('a', 'a'), {})
||||e: pat_match (('a', 'a'), {}) = True
||||s: pat_match ((['?X'], ['vacation']), {})
|||||s: pat_match (('?X', 'vacation'), {})
|||||e: pat_match (('?X', 'vacation'), {}) = True
|||||s: pat_match (([], []), {})
|||||e: pat_match (([], []), {}) = True
||||e: pat_match ((['?X'], ['vacation']), {}) = True
|||e: pat_match ((['a', '?X'], ['a', 'vacation']), {}) = True
||e: pat_match ((['need', 'a', '?X'], ['need', 'a', 'vacation']), {}) = True
|e: pat_match ((['I', 'need', 'a', '?X'], ['I', 'need', 'a', 'vacation']), {}) = True


### pattern matching v2
```lisp
(defun pat-match (pattern input &optional (bindings no-bindings))
  (cond ((eq bindings fail) 
          fail)

        ((variable-p pattern) (match-variable pattern input bindings))

        ((eql pattern input) bindings)

        ((and (consp pattern) (consp input)) 
          (pat-match (cdr pattern) (cdr input) (pat-match (car pattern) (car input) bindings)))
          
        (t fail)
        
    ))

(defun match-variable (var input bindings)
  (let ((binding (get-binding var bindings)))
    (cond ((not binding) (extend-bindings var input bindings))
          ((equal input (binding-val binding)) bindings)
          (t fail))))
```

In [4]:
@debug_tools.debug_print
def pat_match(pattern:list, _input:list):
    #"Does pattern match input? Any variable can match anything."
    if (pattern == [] and _input == []):
        return True
    
    elif variable_p(pattern):
        return True
    else:
        if atom(pattern) or atom(_input):
            return pattern == _input
        else:
            return pat_match(pattern[0], _input[0]) and pat_match(pattern[1:], _input[1:])

n: [1, 2, 3]
[1, 2, 3, 2, 3]
n: [1, 2]
[1, 2, 2, 3]


In [5]:
a = [1]
a.pop()

1

In [6]:
if [1] and [1]:
    print(1)

1
